# 求df的差集
给定两个 dataframe, 称之为 df1 df2, 求 df1 df2 中, 当 key 相同时, 其他列差值小于d的所有行.

推荐的方法是: 先merge数据集, 然后通过 filter 或者 sql 等任意方式比较行内差值即可.
- sql示例: `select * from t1 where a_c1 != a_c2 and b_c1 != b_c2`
- 对于 pandas, `df[df.a_c1!=df.a_c2]` 即可filter数据, 或者使用自由度更高的 apply 函数.

完整示例参考 [diff_two_csv.py](/tools/2-pandas/code/diff_two_csv.py)

## 其他方法
有一些其他思路也可以解决问题, 写到这里供参考. 但是我觉得不如推荐的方法

---
去重, 求交补集

当只需要去掉重复值时, 可以考虑使用交补集的方式. 在pandas中直接使用 drop_duplicates 即可
- 示例 `df.drop_duplicates(['date', 'name', 'score'], keep=False, inplace=True)`

也是一个方法, 但是经常在去重后, 还需要 merge dataframe, 以方便查看.

---
sql思维的方法

使用 concat 合并数据集, 这时类似于在数据库表中的情况(但这个不如merge优雅, 效率高), 然后使用sql找出表中相同key不同value的数据.

方式如下. 首先, 使用 concat 合并数据集, 注册为表, 然后
1. 自己 join 自己, join 条件添加限制直接的出结; 或不加限制, 变成merge的情况.
2. 使用子查询
3. groupby 后, 在 having中 判断 最大值!=最小值.

In [12]:
import pandas as pd

def output(df):
    print(df.head())

In [10]:
def diff_two_df(df1, df2):
    'merge 数据集, 然后 filter'
    df = df1.merge(df2, on=['date', 'name'], how='outer', suffixes=('_d1', '_d2'))
    df = df[df.score_d1!=df.score_d2]
    return df

In [16]:
# 推荐方法, 使用 apply
def diff_two_df(df1, df2):
    'merge 数据集, 然后使用 apply filter'
    df = df1.merge(df2, on=['date', 'name'], how='outer', suffixes=('_d1', '_d2'))
    def diff(x):
        if x.score_d1!=x.score_d2:
            return x
    
    df = df.apply(diff,axis=1).dropna()
    return df

In [20]:
df1 = pd.DataFrame ({'date' : ['2020-02-03','2020-02-03','2020-02-03','2020-02-02','2020-02-02','2020-02-02'], \
                     'name' : ['g','m','z','g','m','z'], \
                     'score' : [3,6,5,5,3,6]})
df2 = pd.DataFrame ({'date' : ['2020-02-03','2020-02-03','2020-02-03','2020-02-02','2020-02-02','2020-02-02'], \
                     'name' : ['g','m','z','g','m','z'], \
                     'score' : [6,2,5,7,3,5]})
df = diff_two_df(df1, df2)
output(df)

         date name  score_c1  score_c2
0  2020-02-03    g         3         6
1  2020-02-03    m         6         2
2  2020-02-02    g         5         7
3  2020-02-02    z         6         5


In [18]:
# 求交补集
# 正是因为有需要合并查看这个需求, 所以这个方法是个败笔
def combine_df(df):
    '在去重后, 还需要 merge dataframe, 以方便查看相同key的值'
    df1 = df[df.source==1]
    df2 = df[df.source==2]
    df1 = df1.drop(columns=['source'])
    df2 = df2.drop(columns=['source'])
    df = df1.merge(df2, on=['date', 'name'], how='outer', suffixes=('_c1', '_c2'))
    return df

def diff_two_df(df1, df2):
    '求交补集'
    df1['source'] = 1
    df2['source'] = 2
    df = pd.concat([df1, df2])
    df.drop_duplicates(['date', 'name', 'score'], keep=False, inplace=True)
    return combine_df(df)  

In [ ]:
# sql思维的方法
# 暂不讨论